# Classification de formes à l'aide d'un réseau de convolution

> Author: Françoise Bouvet (IJCLab, CNRS)  
> Email: <francoise.bouvet@ijclab.in2p3.fr>

1. [Introduction](#Introduction)
2. [Préparation des données](#Préparation-des-données)
3. [Structure du réseau](#Structure-du-réseau)
4. [Apprentissage](#Apprentissage)
5. [Evaluation](#Evaluation)
6. [Exercice](#Exercice)

## Introduction

L'objectif de ce TP est d'améliorer la reconnaissance des 5 formes géométriques avec un réseau de convolution.  

Dans un premier temps, votre travail consiste à générer un réseau de convolution basique et à le tester. Pour cela, vous devez remplacer les ... par le code adéquat. Vous pourrez ensuite le compléter pour essayer d'améliorer les résultats.  

Les données sont dans le répertoire data-shape. Il contient un sous-répertoire "train" dans lequel sont regroupés les fichiers d'apprentissage et un sous-répertoire "test" dans lequel sont regroupés les fichiers tests. Cinq formes différentes sont à distinguer : carré, rectangle, cercle, ellipse et triangle. Les images sont représentées sur 2 niveaux de gris, 0 pour l'intérieur des formes et 255 à l'extérieur. Elles sont de taille 64*64 pixels.  

Les images sont issues de la base de données de la plateforme [kaggle](https://www.kaggle.com/)  

Pour vous aider, vous trouverez des informations complémentaires sur la librairie [keras](https://keras.io/getting_started/)

## Préparation des données

La fonction de lecture des fichiers est fournie dans utils.py . Elle retourne un tableau avec les données d'entrée et un avec celles de sortie. Les données d'entrée sont ensuite normalisées sur [0,1] et celles en sortie sont transformées en variables catégorielles. Pour finir, les données sont mélangées : on utilise la fonction shuffle de numpy sur la liste des indices.

In [ ]:
import numpy as np
from utils import lecture_shape_1channel

rep_data = "../datasets/data_shape/"
lst_shape = ['circle', 'ellipse', 'rectangle', 'square', 'triangle']

# Read input data and transform output into one hot encoding
input_train_raw, output_train_raw = lecture_shape_1channel(rep_data + "train/", "*.png", lst_shape)

if input_train_raw is None or not np.any(input_train_raw):
    print(f'Aucun fichier {extension} trouvé dans {dir}')

In [ ]:
from keras.utils import to_categorical

# Normalize input data
input_train = input_train_raw.astype('float32') / 255.
# Transform output into one hot encoding
output_train = to_categorical(output_train_raw)

# Shuffle input data
ind = np.arange(0, np.shape(input_train)[0])
np.random.shuffle(ind)

# Apply to data ; 
input_train = input_train[ind]
output_train = output_train[ind]

print(f'Il y a {input_train.shape[0]} échantillons')

## Structure du réseau
Le modèle est défini par ajout successif des couches. Les données en entrée sont sous forme de tableau (64, 64, 1). On rajoute successivement la couche de convolution et la couche de pooling. 

Pour la couche de convolution, spécifier : 
 
* le nombre de filtres
* la taille du noyau de chaque filtre : kernel_size=... (typiquement (3, 3) ou (5, 5))
* la fontion d'activation : activation='...' (typiquement 'relu')
* le type de pagging : padding='...' (typiquement 'same')
* pour la première couche de convolution, spécifier input_shape est indispensablen si on veut utiliser la fonction summmary() function ensuite

Pour le pooling, ajouter une couche de type MaxPooling2D ou AveragePooling2D et spécifier 
* la taille du regroupement en x et y : pool_size='...' (typiquement (2,2)) 
* le pas de déplacement : stride='...' (facultatif, par défaut la taille du pooling) 

Les données de sortie de la dernière couche de convolution sont transférées dans un vecteur 1D, puis on connecte les couches du perceptron multicouche. Etant donné qu'on a choisi un encodage de type one-hot pour les données en sortie, le nombre de neurones dans la couche de sortie est le nombre de classes et la fonction d'activation est softmax.

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout

# Network definition : add successive layers
model = Sequential()

# Convolution layers
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same', input_shape=(64, 64, 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
#
model.add(Conv2D(128, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))
#
model.add(Conv2D(256, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))
#
model.add(Conv2D(256, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Reshape input 1D vector
model.add(Flatten())

# Full connected layer (MLP)
model.add(Dense(256, activation='relu'))
model.add(Dense(256, activation='relu'))

# # Output layer
model.add(Dense(5, activation='softmax'))

### Configuration complète
Une fois la structure définie, le modèle doit être complètement configuré en spécifiant la fonction de coût, la métrique et l'algorithme d'optimisation.  
On spécifie donc :
* la fonction de coût utilisée pour calculer l'erreur à rétropropager : loss='...' ; (ex ici : 'categorical_crossentropy')
* l'algorithme d'optimisation : optimizer='...' (ex : 'adam', 'adamax', 'rmsprop', 'sgd')
* la liste des métriques évaluées à titre indicatif à chaque étape : metrics=['...'] (ex ici : 'categorical_accuracy') 

On peut ensuite vérifier avec summary() que le modèle est conforme à ce que l'on souhaite.

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['categorical_accuracy'])

# Display the model
model.summary()

## Apprentissage
L'apprentissage est réalisé avec la fonction fit(...)  
On spécifie : 
* le tableau des données en entrée
* le tableau des données souhaitées en sortie
* le nombre d'époques epochs=...  (commencer avec une petite valeur)  

Facultatif :
* la proportion des données utilisées pour la validation validation_split=... (typiquement entre 0.1 et 0.4)
* la taille des batch batch=... (taille des paquets utilisés pour l'apprentissage ; typiquement 16, 32, 64)
* les fonctions de callback
* verbose= 0, 1 ou 2 en fonction de ce que l'on souhaite afficher au cours de l'apprentissage

In [ ]:
history = model.fit(input_train, output_train,
                    epochs=20,
                    validation_split=0.2,
                    batch_size=32,
                    verbose=2)

#### La fonction "draw_history" fournie dans utils.py permet de visualiser l'évolution de l'apprentissage.

In [ ]:
from utils import draw_history

draw_history(history)

## Evaluation

Dans un premier temps, il faut **lire les données tests** et les mettre au bon format. Cette étape est similaire à l'étape de préparation des données. Seule différence, les données sont dans le répertoire "test" et non "train".
On peut évaluer le score du modèle sur l'ensemble des données test, et on peut prédire la classe échantillon par échantillon.

In [ ]:
# Read test data
input_test, output_test = lecture_shape_1channel(rep_data + "test/", "*.png", lst_shape)
# Normalize test data
input_test = input_test.astype('float32') / 255.
# Transform output into one hot encoding
output_test = to_categorical(output_test)

# Test set shuffle only aims to display samples from all the classes
ind = np.arange(np.shape(input_test)[0])
np.random.shuffle(ind)
input_test = np.array(input_test)[ind]
output_test =np.array(output_test)[ind]

# Evaluate the model ; the two parameters are the input_test array and the output_test array
sum_score = model.evaluate(input_test, output_test)
print("Data test : loss %.3f accuracy %.3f" % (sum_score[0], sum_score[1]))

# Prediction ; the input parameter is the input_test array ; return value is the prediction array
output_predict = model.predict(input_test)

#### Affichage des résultats sous forme de mosaïque d'images 
On peut utiliser la fonction draw_multiple_images(input_test, output_test, output_predict, lst_shape)

In [ ]:
from utils import draw_multiple_images

# Display the test images and the predicted class
n=min(40, np.shape(input_test)[0])
nb_col = 8
draw_multiple_images(input_test[0:n], output_test[0:n], output_predict[0:n], lst_shape, nb_col, np.shape(input_test)[1:])